In [1]:
import calibrate
import simulate
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize
import simulate

In [2]:
class HawkesProcessCalibrator:
    def __init__(self, events):
        self.events = events

    def calibrate(self):
        res = minimize(fun=self.nll, x0=[1,1,2], bounds=[(1e-10, None), (1e-10, None), (1e-10, None)], method='L-BFGS-B')
        return res.x
    
    def nll(self, theta):
        # source: https://www.ism.ac.jp/editsec/aism/pdf/031_1_0145.pdf
        mu, alpha, beta = theta
        kappa = alpha/beta
        t_n = self.events[-1]
        n = len(self.events)
        ll = t_n - mu * t_n
        for i in range(n):
            ll = ll - kappa * (1 - np.exp(-beta*(t_n - self.events[i])))

        r_array = np.zeros(n)
        for i in range(1, n):
            r_array[i] = np.exp(-beta * (self.events[i] - self.events[i - 1])) * (1 + r_array[i - 1])

        for i in range(n):
            ll += np.log(mu + alpha*r_array[i])
        return -ll

In [21]:
mu, alpha, beta = 0.1, 1, 1.2
hwk_proc = simulate.HawkesProcess(mu, alpha, beta)
T = 1000
events = hwk_proc.simulate(T)
hwk_proc.plot(events)
n_events = len(events)
print("Number of events: ", n_events)

cal = HawkesProcessCalibrator(events)
mu, alpha, beta = cal.calibrate()
print("Calibrated mu, alpha, beta: ", mu, alpha, beta)
print("True mu, alpha, beta: ", hwk_proc.mu, hwk_proc.alpha, hwk_proc.beta)

print("log-likelihood of true parameters: ", -cal.nll([hwk_proc.mu, hwk_proc.alpha, hwk_proc.beta]))
print("log-likelihood of calibrated parameters: ", -cal.nll([mu, alpha, beta]))

AttributeError: 'HawkesProcess' object has no attribute 'plot'